### Step 1: Importing librarires and loading data 

In [1]:
# Essentials
import pandas as pd
import numpy as np

# Library required to change date format
from datetime import datetime

# Ignore all FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Loading different brokers dataframes
angel_one = pd.read_json('/Users/shounak/Documents/Cuvette/InterviewProj/AngelBrokingScripMaster.json')
fyers_capital_market = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/FyersCapitalMarket.csv')
fyers_equity_derivatives = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/fyersED.csv')
fyers_BSE_capital_market = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Fyers_BSE_CM.csv')
fyers_MCX_commodity = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Fyers_MCX_COM.csv')
fyers_nse_currency_derivatives = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Fyers_NSE_CD.csv')
kotakcash = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/kotakcash.txt', delimiter='|')
kotak_fao = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Kotakf&o.txt', delimiter='|')
zerodha = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/zerodha.csv')
icici_bse = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/SecurityMasterICICI/BSEScripMaster.txt',
                      delimiter=',')
icici_nse = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/SecurityMasterICICI/NSEScripMaster.txt',
                      delimiter=',')

In [3]:
# Understanding basic data about all broker dataframes
datafiles = [angel_one, fyers_capital_market, fyers_equity_derivatives, fyers_BSE_capital_market, 
             fyers_MCX_commodity, fyers_nse_currency_derivatives, kotakcash, kotak_fao, zerodha, icici_bse, 
             icici_nse]
file_names = ['angel_one', 'fyers_capital_market', 'fyers_equity_derivatives', 'fyers_BSE_capital_market', 
             'fyers_MCX_commodity', 'fyers_nse_currency_derivatives', 'kotakcash', 'kotak_fao', 'zerodha',
             'icici_bse', 'icici_nse']

for name, df in zip(file_names, datafiles):
    print(name)
    print(df.head(), end='\n\n')

angel_one
   token        symbol       name expiry  strike  lotsize instrumenttype  \
0   6293    GULPOLY-BL    GULPOLY             -1        1                  
1  24051    739UK30-SG    739UK30             -1      100                  
2   3475    THERMAX-EQ    THERMAX             -1        1                  
3   3523     69MZ33-SG     69MZ33             -1      100                  
4  17996  PGIMCSR2G-MF  PGIMCSR2G             -1        1                  

  exch_seg  tick_size  
0      NSE          1  
1      NSE          1  
2      NSE          5  
3      NSE          1  
4      NSE          1  

fyers_capital_market
      10100000001     GOLDSTAR POWER LIMITED  0  11250  0.05  INE405Y01021  \
0    101000000010         ABAN OFFSHORE LTD.  0      1  0.01  INE421A01028   
1   1010000000100  AMARA RAJA ENERGY MOB LTD  0      1  0.05  INE885A01032   
2  10100000001000          SDL MH 6.56% 2032  5    100  0.01  IN2220200306   
3  10100000001001    SEC RED NCD 9.40% SR. V  2      1 

In [4]:
# Function to convert date format for angel_one 
def convert_date(date_str):
    if date_str:  # Check if the string is not empty
        date_obj = datetime.strptime(date_str, '%d%b%Y')
        return date_obj.strftime('%Y-%m-%d')
    return None  # Return None for empty strings

# Apply the conversion function to the 'expiry' column
angel_one['expiry'] = angel_one['expiry'].apply(convert_date)

### Step 2: Concatinating dataframes of same broker into single dataframe and preprocessing the data

In [5]:
#Concatinating Fyers dataframes into single dataframe and dropping columns
fyers_df = [fyers_capital_market, fyers_equity_derivatives, fyers_BSE_capital_market, fyers_MCX_commodity, 
            fyers_nse_currency_derivatives]

# Define the target columns for Fyers DataFrames
fyers_columns = [
    "instrument_id", "name", "lot_size", "tick_size", "min_order_qty", "isin", "trading_hours",
    "expiry_date", "placeholder1", "exchange_symbol", "other_info", "close_price", "status",
    "symbol", "price_factor", "price_multiplier", "market_segment", "unique_code", 
    "placeholder2", "turnover", "open_interest"
]

# Columns to drop
drop_columns = [
    'placeholder1', 'placeholder2', 'turnover', 'open_interest', 'status', 
    'min_order_qty', 'trading_hours', 'close_price', 'price_factor', 'isin', 
    'other_info', 'price_multiplier', 'market_segment', 'unique_code'
]

# Process each Fyers DataFrame
for df in fyers_df:
    df.columns = fyers_columns  # Rename columns
    df.drop(columns=drop_columns, inplace=True)  # Drop unnecessary columns
    
    # Extract the part before the colon
    df['exchange'] = df['exchange_symbol'].str.split(':').str[0]
    df.drop(columns=['exchange_symbol'], inplace = True)
    
# Concatenate all DataFrames vertically
fyers = pd.concat(fyers_df, axis=0, ignore_index=True)
fyers.drop_duplicates(inplace=True)

fyers

,instrument_id,name,lot_size,tick_size,expiry_date,symbol,exchange
0,101000000010,ABAN OFFSHORE LTD.,0,1,2024-10-24,ABAN,NSE
1,1010000000100,AMARA RAJA ENERGY MOB LTD,0,1,2024-10-24,ARE&M,NSE
2,10100000001000,SDL MH 6.56% 2032,5,100,2024-10-24,656MH32,NSE
3,10100000001001,SEC RED NCD 9.40% SR. V,2,1,2024-10-24,94SFL28,NSE
4,10100000001004,SDL AP 6.79% 2034,5,100,2024-10-24,679AP34,NSE
...,...,...,...,...,...,...,...
120204,10122509269586,JPYINR 25 Sep 26 54.75 CE,19,1,2024-10-25,JPYINR,NSE
120205,10122509269587,JPYINR 25 Sep 26 54.75 PE,19,1,2024-10-25,JPYINR,NSE
120206,10122509269588,JPYINR 25 Sep 26 55 CE,19,1,2024-10-25,JPYINR,NSE
120207,10122509269597,JPYINR 25 Sep 26 55 PE,19,1,2024-10-25,JPYINR,NSE


In [6]:
# Concatinating Kotak dataframes into single dataframe and dropping columns

# Columns to drop
kotakcash.drop(columns=['name', 'lastPrice', 'OptionType', 'multiplier', 'exchangeToken'], inplace=True)
kotak_fao.drop(columns=['name', 'lastPrice', 'optionType', 'multiplier', 'exchangeToken'], inplace=True)

kotak_df = [kotakcash, kotak_fao]

# Concatenate all DataFrames vertically
kotak = pd.concat(kotak_df, axis=0, ignore_index=True)
kotak.drop_duplicates(inplace=True)

kotak

,instrumentToken,instrumentName,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin
0,9516,20MICRONS,0,0.0,0.05,1,EQ,CASH,NSE,INE144J01027
1,43865,3IINFOLTD,0,0.0,0.05,1,EQ,CASH,NSE,INE748C01038
2,891,3MINDIA,0,0.0,0.05,1,EQ,CASH,NSE,INE470A01017
3,607,3PLAND,0,0.0,0.05,1,EQ,CASH,NSE,INE105C01023
4,13140,5PAISA,0,0.0,0.05,1,EQ,CASH,NSE,INE618L01018
...,...,...,...,...,...,...,...,...,...,...
113731,17583,NIFTY,30MAR23,19000.0,0.05,50,OS,FO,NSE,NaN
113732,28592,NIFTY,30MAR23,19000.0,0.05,50,OS,FO,NSE,NaN
113733,28593,NIFTY,30MAR23,20000.0,0.05,50,OS,FO,NSE,NaN
113734,28594,NIFTY,30MAR23,20000.0,0.05,50,OS,FO,NSE,NaN


In [7]:
# Dropping some columns in zerodha dataframe

# Columns to drop
zerodha.drop(columns=['exchange_token', 'last_price'], inplace=True)
zerodha

,instrument_token,tradingsymbol,name,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,264741126,EURINR24OCTFUT,EURINR,2024-10-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
1,265969926,EURINR24NOVFUT,EURINR,2024-11-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
2,268449030,EURINR24DECFUT,EURINR,2024-12-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
3,269248262,EURINR25JANFUT,EURINR,2025-01-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
4,270018310,EURINR25FEBFUT,EURINR,2025-02-26,0.0,0.0025,1,FUT,BCD-FUT,BCD
...,...,...,...,...,...,...,...,...,...,...
89811,8150273,SREEL,SREELEATHERS,NaN,0.0,0.0500,1,EQ,NSE,NSE
89812,8183297,863NHB29-N2,NaN,NaN,0.0,0.0100,1,EQ,NSE,NSE
89813,8183553,863NHB34-N3,NaN,NaN,0.0,0.0100,1,EQ,NSE,NSE
89814,8184065,888NHB29-N5,NaN,NaN,0.0,0.0100,1,EQ,NSE,NSE


In [8]:
# Preprocessing data in icici

# Removing spaces and double inverted commas from the column names
icici_nse.columns = icici_nse.columns.str.strip().str.replace('"', '').str.strip()

# Columns to drop 
icici_bse.drop(columns=[
    '52WeeksHigh', '52WeeksLow', 'LifeTimeHigh', 'LifeTimeLow', 'HighDate', 'LowDate',
    'MarginPercentage', 'ShortName', 'Series', 'ScripCode', 'MarketLot', 'BCastFlag',
    'AVMBuyMargin', 'AVMSellMargin', 'ScripID', 'ScripName', 'GroupName', 'NdFlag',
    'NDSDate', 'NDEDate', 'SuspStatus', 'avmflag', 'SuspensionReason', 'Suspensiondate',
    'DateOfListing', 'DateOfDeListing', 'IssuePrice', 'FaceValue', 'ExchangeCode', 'CompanyName'
], inplace=True)

icici_nse.drop(columns=[
    'ShortName', 'Series', 'CompanyName', 'DateOfListing', 'DateOfDeListing', 'IssuePrice',
    'FaceValue', '52WeeksHigh', '52WeeksLow', 'LifeTimeHigh', 'LifeTimeLow', 'HighDate',
    'LowDate', 'PermittedToTrade', 'IssueCapital', 'WarningPercent', 'FreezePercent',
    'CreditRating', 'IssueRate', 'IssueStartDate', 'InterestPaymentDate', 'IssueMaturityDate',
    'BoardLotQty', 'Name', 'ListingDate', 'ExpulsionDate', 'ReAdmissionDate', 'RecordDate',
    'NoDeliveryStartDate', 'NoDeliveryEndDate', 'MFill', 'AON', 'ParticipantInMarketIndex',
    'BookClsStartDate', 'BookClsEndDate', 'EGM', 'AGM', 'Interest', 'Bonus', 'Rights',
    'Dividends', 'LocalUpdateDateTime', 'DeleteFlag', 'Remarks', 'NormalMarketStatus',
    'OddLotMarketStatus', 'SpotMarketStatus', 'AuctionMarketStatus', 'NormalMarketEligibility',
    'OddLotlMarketEligibility', 'SpotMarketEligibility', 'AuctionlMarketEligibility', 'Symbol', 'ExchangeCode',
    'MarginPercentage', 'ExpiryDate', 'InstrumentType'
], inplace=True)

# Standardising column to concat further
mappings = {
    'Token': 'instrument_id',
    'LotSize': 'lot_size',
    'Lotsize': 'lot_size',
    'ticksize': 'tick_size',
    'TickSize': 'tick_size',
    'ISINCode': 'isin'
}

icici_df = [icici_bse, icici_nse]
for df in icici_df:
    df.rename(columns=mappings, inplace=True)

# Concatenated two dataframes into single one
icici = pd.concat(icici_df, axis=0, ignore_index=True)
icici

,instrument_id,tick_size,lot_size,isin
0,800078,115.10,1,INY019980013
1,800004,110.20,1,IN0019730012
2,800032,112.25,1,IN0019880023
3,800049,114.40,1,INY019930083
4,800081,118.60,1,INY019980054
...,...,...,...,...
15692,SAYOFS,0.01,1,INE000000102
15693,X25889X,0.01,1,INE840D01015
15694,X25903X,0.01,1,INE200F01017
15695,X5572,0.01,1,INF579M01019


In [9]:
# Standard column names mapping
column_mappings = {
    'token': 'instrument_id',
    'instrumentToken': 'instrument_id',
    'instrument_token': 'instrument_id',
    'expiry': 'expiry_date', 
    'exchange_token': 'exchange_token',
    'symbol': 'tradingsymbol',
    'instrumentName': 'name',
    'lastPrice': 'last_price',
    'last_price': 'last_price',
    'strike': 'strike_price',
    'lotsize': 'lot_size',
    'lotSize': 'lot_size',
    'tickSize': 'tick_size',
    'instrumenttype': 'instrument_type',
    'instrumentType': 'instrument_type',
    'exch_seg': 'exchange',
    'segment': 'market_segment'
}

broker_dfs = [angel_one, fyers, kotak, zerodha, icici]
brokers = ['angel_one', 'fyers', 'kotak', 'zerodha', 'icici']

for name, df in zip(brokers, broker_dfs):
    df.rename(columns=column_mappings, inplace=True)
    print(name)
    print(df.columns, end='\n\n')

angel_one
Index(['instrument_id', 'tradingsymbol', 'name', 'expiry_date', 'strike_price',
       'lot_size', 'instrument_type', 'exchange', 'tick_size'],
      dtype='object')

fyers
Index(['instrument_id', 'name', 'lot_size', 'tick_size', 'expiry_date',
       'tradingsymbol', 'exchange'],
      dtype='object')

kotak
Index(['instrument_id', 'name', 'expiry_date', 'strike_price', 'tick_size',
       'lot_size', 'instrument_type', 'market_segment', 'exchange', 'isin'],
      dtype='object')

zerodha
Index(['instrument_id', 'tradingsymbol', 'name', 'expiry_date', 'strike_price',
       'tick_size', 'lot_size', 'instrument_type', 'market_segment',
       'exchange'],
      dtype='object')

icici
Index(['instrument_id', 'tick_size', 'lot_size', 'isin'], dtype='object')



In [10]:
# Created a function to clean dataframes
def clean_dataframe(df):
    # Replace empty strings in all columns with NaN
    df.replace('', np.nan, inplace=True)
    
    # Remove duplicate rows
    df.drop_duplicates(inplace=True)
    
    # Drop columns with more than 50% missing values
    threshold = len(df) * 0.5
    df.dropna(thresh=threshold, axis=1, inplace=True)
    
    # Fill NaN values for categorical columns with the mode
    catg_cols = df.select_dtypes(include=['object']).columns
    df[catg_cols] = df[catg_cols].fillna(df[catg_cols].mode().iloc[0])
    
    # Fill NaN values for numeric columns with the mode
    num_cols = df.select_dtypes(include=['number']).columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].mode().iloc[0])

# Apply the function to each dataframe in the list
for name, df in zip(brokers, broker_dfs):
    print(name)
    print(df.columns, end='\n\n')

angel_one
Index(['instrument_id', 'tradingsymbol', 'name', 'expiry_date', 'strike_price',
       'lot_size', 'instrument_type', 'exchange', 'tick_size'],
      dtype='object')

fyers
Index(['instrument_id', 'name', 'lot_size', 'tick_size', 'expiry_date',
       'tradingsymbol', 'exchange'],
      dtype='object')

kotak
Index(['instrument_id', 'name', 'expiry_date', 'strike_price', 'tick_size',
       'lot_size', 'instrument_type', 'market_segment', 'exchange', 'isin'],
      dtype='object')

zerodha
Index(['instrument_id', 'tradingsymbol', 'name', 'expiry_date', 'strike_price',
       'tick_size', 'lot_size', 'instrument_type', 'market_segment',
       'exchange'],
      dtype='object')

icici
Index(['instrument_id', 'tick_size', 'lot_size', 'isin'], dtype='object')



In [11]:
# Renaming columns to add broker-specific prefix
angel_one = angel_one.add_prefix('angel_one_')
fyers = fyers.add_prefix('fyers_')
kotak = kotak.add_prefix('kotak_')
zerodha = zerodha.add_prefix('zerodha_')
icici = icici.add_prefix('icici_')

### Step 3: Mapping all dataframes based on trading-symbol, exchange, isin, instrument_id  and name as keys

In [12]:
# Dropping duplicates in Zerodha and Fyers based on the trading symbol
zerodha_unique = zerodha.drop_duplicates(subset=['zerodha_tradingsymbol'])
fyers_unique = fyers.drop_duplicates(subset=['fyers_tradingsymbol'])

# Merging Zerodha and Fyers on trading symbol with 'inner' join to keep matched rows
mdf = zerodha_unique.merge(fyers_unique, left_on='zerodha_tradingsymbol', right_on='fyers_tradingsymbol',
                           how='inner')

# Drop duplicates in Angel One to maintain unique entries
angel_one_unique = angel_one.drop_duplicates(subset=['angel_one_tradingsymbol', 'angel_one_exchange'])

# Merging the result with Angel One on trading symbol and exchange with 'inner' join
mdf = mdf.merge(
    angel_one_unique, 
    left_on=['zerodha_tradingsymbol', 'zerodha_exchange'], 
    right_on=['angel_one_tradingsymbol', 'angel_one_exchange'], 
    how='inner'
)

# Cleaning up the merged dataFrame 
mdf.dropna(subset=['zerodha_instrument_id', 'fyers_instrument_id', 'angel_one_instrument_id'], inplace=True)

mdf


,zerodha_instrument_id,zerodha_tradingsymbol,zerodha_name,zerodha_expiry_date,zerodha_strike_price,zerodha_tick_size,zerodha_lot_size,zerodha_instrument_type,zerodha_market_segment,zerodha_exchange,...,fyers_exchange,angel_one_instrument_id,angel_one_tradingsymbol,angel_one_name,angel_one_expiry_date,angel_one_strike_price,angel_one_lot_size,angel_one_instrument_type,angel_one_exchange,angel_one_tick_size
0,128000516,ABB,ABB INDIA,NaN,0.0,0.05,1,EQ,BSE,BSE,...,NSE,500002,ABB,ABB,None,-1,1,,BSE,5
1,128000772,AEGISLOG,AEGIS LOGISTICS,NaN,0.0,0.05,1,EQ,BSE,BSE,...,NSE,500003,AEGISLOG,AEGISLOG,None,-1,1,,BSE,5
2,128002052,ARE&M,AMARA RAJA ENERGY & MOBILITY L,NaN,0.0,0.05,1,EQ,BSE,BSE,...,NSE,500008,ARE&M,ARE&M,None,-1,1,,BSE,5
3,128002308,AMBALALSA,AMBALAL SARABHAI ENTERPRISES L,NaN,0.0,0.01,1,EQ,BSE,BSE,...,BSE,500009,AMBALALSA,AMBALALSA,None,-1,1,,BSE,1
4,128003076,ANDHRAPET,ANDHRA PETROCHEMICALS,NaN,0.0,0.01,1,EQ,BSE,BSE,...,BSE,500012,ANDHRAPET,ANDHRAPET,None,-1,1,,BSE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11828,286985,SNXT50,BSE INDEX SNXT50,NaN,0.0,0.00,0,EQ,INDICES,BSE,...,BSE,99919083,SNXT50,S&P BSE SENSEX NEXT 50,None,0,1,AMXIDX,BSE,0
11829,287241,TECK,BSE INDEX TECK,NaN,0.0,0.00,0,EQ,INDICES,BSE,...,BSE,99919011,TECK,S&P BSE TECK,None,0,1,AMXIDX,BSE,0
11830,287497,TELCOM,BSE INDEX TELCOM,NaN,0.0,0.00,0,EQ,INDICES,BSE,...,BSE,99919087,TELCOM,S&P BSE TELECOM,None,0,1,AMXIDX,BSE,0
11831,292105,MCXMETLDEX,MCXMETLDEX,NaN,0.0,0.00,1,EQ,INDICES,MCX,...,MCX,99920004,MCXMETLDEX,MCXMETLDEX,None,0,1,AMXIDX,MCX,0


In [13]:
# Filtering kotak and icici for only required rows and columns
kotak_filtered = kotak[kotak['kotak_market_segment'] == 'CASH']
icici_filtered = icici[icici['icici_tick_size'] > 0]  

# Merging the cleaned dataframes, keeping only unique entries 
mdf2 = icici_filtered.merge(kotak_filtered[['kotak_instrument_id', 'kotak_name', 'kotak_isin', 'kotak_exchange']],
    left_on='icici_isin', right_on='kotak_isin', how='inner').drop_duplicates(subset=['icici_isin', 
                                                                                      'kotak_instrument_id'])

# Filtering out rows with missing data in critical columns
mdf2 = mdf2.dropna(subset=['icici_instrument_id', 'kotak_instrument_id'])

mdf2


,icici_instrument_id,icici_tick_size,icici_lot_size,icici_isin,kotak_instrument_id,kotak_name,kotak_isin,kotak_exchange
0,533022,290.75,1,INE144J01027,9516,20MICRONS,INE144J01027,NSE
1,533022,290.75,1,INE144J01027,9515,20MICRONS,INE144J01027,BSE
2,532628,32.30,1,INE748C01038,43865,3IINFOLTD,INE748C01038,NSE
3,532628,32.30,1,INE748C01038,4285,3IINFOTECH,INE748C01038,BSE
4,523395,39958.25,1,INE470A01017,891,3MINDIA,INE470A01017,NSE
...,...,...,...,...,...,...,...,...
8660,9539,0.01,1,INE732S01012,12337,SAKAR,INE732S01012,NSE
8665,9576,0.01,1,INE263W01010,15877,MAHESHWARI,INE263W01010,NSE
8695,9847,0.01,1,INF204KB18I3,33663,NETFNV20,INF204KB18I3,NSE
8714,9937,0.01,1,INF666M01FS5,11913,IBMFNIFTY,INF666M01FS5,NSE


In [14]:
# Merging the previously merged dataframes 'mdf' and 'mdf2' using inner join on name column
unified = mdf.merge(mdf2, left_on = ['angel_one_name'], 
                    right_on = ['kotak_name'], how='inner')

# Displaying the final unified/mapped csv file
unified

,zerodha_instrument_id,zerodha_tradingsymbol,zerodha_name,zerodha_expiry_date,zerodha_strike_price,zerodha_tick_size,zerodha_lot_size,zerodha_instrument_type,zerodha_market_segment,zerodha_exchange,...,angel_one_exchange,angel_one_tick_size,icici_instrument_id,icici_tick_size,icici_lot_size,icici_isin,kotak_instrument_id,kotak_name,kotak_isin,kotak_exchange
0,128000516,ABB,ABB INDIA,NaN,0.0,0.05,1,EQ,BSE,BSE,...,BSE,5,500002,8071.15,1,INE117A01022,720,ABB,INE117A01022,NSE
1,128000516,ABB,ABB INDIA,NaN,0.0,0.05,1,EQ,BSE,BSE,...,BSE,5,500002,8071.15,1,INE117A01022,719,ABB,INE117A01022,BSE
2,128003076,ANDHRAPET,ANDHRA PETROCHEMICALS,NaN,0.0,0.01,1,EQ,BSE,BSE,...,BSE,1,500012,101.79,1,INE714B01016,764,ANDHRAPET,INE714B01016,BSE
3,128005124,BOMDYEING,BOMBAY DYEING & MFG.CO.LTD.,NaN,0.0,0.05,1,EQ,BSE,BSE,...,BSE,5,500020,263.45,1,INE032A01023,900,BOMDYEING,INE032A01023,NSE
4,128005124,BOMDYEING,BOMBAY DYEING & MFG.CO.LTD.,NaN,0.0,0.05,1,EQ,BSE,BSE,...,BSE,5,500020,263.45,1,INE032A01023,15842,BOMDYEING,INE032A01023,BSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4512,204872196,SGBJAN26,SOVEREIGN GOLD BOND 2018 SERIE,NaN,0.0,0.01,1,EQ,BSE,BSE,...,BSE,1,800282,8637.38,1,IN0020170166,12275,SGBJAN26,IN0020170166,BSE
4513,204873988,SGBNOV26,SOVEREIGN GOLD BOND 2019 SERIE,NaN,0.0,0.01,1,EQ,BSE,BSE,...,BSE,1,800289,8651.79,1,IN0020180314,12282,SGBNOV26,IN0020180314,BSE
4514,204874500,SGBJAN27,SOVEREIGN GOLD BOND 2019 SERIE,NaN,0.0,0.01,1,EQ,BSE,BSE,...,BSE,1,800291,8635.25,1,IN0020180462,12289,SGBJAN27,IN0020180462,BSE
4515,204881924,SGBMAY28,SOVEREIGN GOLD BONDS 2020-21 S,NaN,0.0,0.01,1,EQ,BSE,BSE,...,BSE,1,800320,8977.10,1,IN0020200088,12446,SGBMAY28,IN0020200088,BSE


In [15]:
# Checking null values
unified.isnull().sum()

zerodha_instrument_id           0
zerodha_tradingsymbol           0
zerodha_name                   13
zerodha_expiry_date          4517
zerodha_strike_price            0
zerodha_tick_size               0
zerodha_lot_size                0
zerodha_instrument_type         0
zerodha_market_segment          0
zerodha_exchange                0
fyers_instrument_id             0
fyers_name                      0
fyers_lot_size                  0
fyers_tick_size                 0
fyers_expiry_date               0
fyers_tradingsymbol             0
fyers_exchange                  0
angel_one_instrument_id         0
angel_one_tradingsymbol         0
angel_one_name                  0
angel_one_expiry_date        4517
angel_one_strike_price          0
angel_one_lot_size              0
angel_one_instrument_type       0
angel_one_exchange              0
angel_one_tick_size             0
icici_instrument_id             0
icici_tick_size                 0
icici_lot_size                  0
icici_isin    

### Step 4: Cleaning Unified dataframe and exporting it

In [16]:
# Dropping columns which are completely empty
unified.drop(columns=['zerodha_expiry_date', 'angel_one_expiry_date'])

# Export the merged DataFrame to a CSV file with the new name
unified.to_csv('/Users/shounak/Documents/Cuvette/FinalProject/Unified_Script_Master.csv', index=False)